In [1]:
# import required libs and packages
from sklearn.feature_extraction.text import CountVectorizer
from keras.models import load_model
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
import pickle
import random
import json

nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mouad\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mouad\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
lemmatizer = WordNetLemmatizer()


context = {}

In [3]:

class Testing:
    def __init__(self):
        # Load the intent file
        self.intents = json.loads(open("./chatbot-code/intents.json").read())

        data = pickle.load(open("training_data", "rb"))
        self.words = data["words"]
        self.classes = data["classes"]
        self.model = load_model("chatbot_model.h5")
        self.ERROR_THRESHOLD = 0.5
        self.ignore_words = list("!@#$%^&*?")

    def clean_up_sentence(self, sentence):
        # Tokenize each sentence (user's query)
        sentence_words = word_tokenize(sentence.lower())
        sentence_words = list(map(lemmatizer.lemmatize, sentence_words))

        sentence_words = list(filter(lambda x: x not in self.ignore_words))

        return set(sentence_words)

    def wordvector(self, sentence):
        cv = CountVectorizer(tokenizer=lambda txt: txt.split())
        sentence_words = " ".join(self.clean_up_sentence(sentence))

        words = " ".join(self.words)

        vectorize = cv.fit([words])

        word_vector = vectorize.transform(
            [sentence_words]).toarray().tolist()[0]

        return (np.array(word_vector))
    
    def classify(self,sentence):
        # Predict to which class  (tag)  user's query belongs to
        
        results = self.model.predict(np.array([self.wordvector(sentence)]))[0]
        
        # Store the class and probability of chat class
        results = list(map(lambda x: [x[0], x[1]],enumerate(results)))
        
        # Accept those class probability which are greater then threshold value 0.5
        
        results = list(filter(lambda x: x[1]> self.ERROR_THRESHOLD, results))
        
        results.sort(key=lambda x: x[1] , reverse = True)
        
        return_lists = []
        
        for i in results:
            return_lists.append((self.classes[i[0]], str(i[1])))
            
        return return_lists
    
    def results(self,sentence,userID):
        
        if sentence.isdecimal():
            if context[userID] == "historydetails":
                return self.classify("ordernumber")
        return self.classify(sentence)
    
    
    def response(self,sentence,userID="Mouad"):
        
        results = self.results(sentence,userID)
        print(sentence,results)
        
        # Store random response to the query
        
        ans=""
        if results:
            for i in self.intents["intents"]:
                # Check if tag  == query's class
                if i['tag'] == results[0][0]:
                    if  "set" in i and not "filter" in i :
                        context[userID] = i['set']
                    if not 'filter' in i :
                        ans = random.choice(i["responses"])
                    if userID in context and 'filter' in i and i['filter']==context[userID]:
                        if "set" in i:
                            context[userID] = i["set"]
                            ans = random.choice(i['responses'])
        results.pop(0)
        # If ans contains some value then return response to user's query else return some message
        
        return ans if ans != "" else "Sorry couldn't understand , I m still learning.\nYou can train me by provinding more data."